In [1]:
import numpy as np
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
np.set_printoptions(threshold=20)


## Calculating the frequency of each attractor period

In [ ]:
sel_param_l = [0.2,0.4,0.6,0.8]
param_pairs = [(1,0),(0,1)] + [(0,ss) for ss in sel_param_l] + [(ss,1-ss) for ss in sel_param_l] + [(ss,1) for ss in sel_param_l]
oords = ['asc','desc']
att_d = {}
att_sizes_d = {}
#num_unchanged = []
#unchanged = []
for rep in range(20):
    for ss,rr in param_pairs:
        for oord in oords:
            if ((ss,rr)!=(0,0)) and ((ss,rr)!=(0,1)):
                attfn = 'attfiles/att_twoparam_%.2f_%.2f_%s_%02d_%d.txt' %(ss,rr,oord,rep,rep)
            elif (ss,rr)==(1,0):
                attfn = 'attfiles/att_twoparam_1.00_0.00_desc_00_0.txt'
            elif (ss,rr)==(0,1):
                attfn = 'attfiles/att_twoparam_0.00_1.00_desc_00_0.txt'
            if not osp.exists(attfn):
                continue
            with open(attfn,encoding='utf-8') as fo:
                lines=fo.readlines()
            for i,line in enumerate(lines):
                if line.startswith('Attractor number:'):
                    att_no = int(line.split(':')[1].split(' ')[0])
                    att_size = int(line.split(':')[-1].strip())
                    att_sizes_d[(oord,ss,rr,rep,att_no)]=att_size
                    if att_size != 1:
                        att = []
                        for k in range(1,att_size+1):
                            state = lines[i-k].strip()
                            att.append(state)
                        att_str = ' '.join(att)
                        att_d[(oord,ss,rr,rep,att_no)]=att_str
att_sizes_ser = pd.Series(att_sizes_d)
att_ser = pd.Series(att_d)
att_ser.to_pickle('att_ser.pkl')

#### Frequency of each attractor period

In [ ]:
attr_freq_ser = att_sizes_ser.value_counts(normalize=True).sort_index()
attr_freq_ser

In [ ]:
def get_attr_stats(anum,sattr,attrsize):
    sattr1 = np.asarray([[int(elt) for elt in bstr] for bstr in sattr.split(' ')])
    HH = [np.sum(np.abs(sattr1[(jj+1)%attrsize]-sattr1[jj])) for jj in range(attrsize)]
    ss = np.sum(sattr1,axis=0)
    cnodes = np.where((ss>0)&(ss<attrsize))[0]
    return anum,cnodes,HH

att_ser = pd.Series(att_d)
hamming_steps2 = {}
changing_nodes2 = {}
changing_node_sets2 = {}
for (order,ss,rr,rep),nonfpcnt in periodic_attrcounts.items():
    print('--->',order,ss,rr,rep)
    sel_attrs = att_ser.xs((order,ss,rr,rep),level=[0,1,2,3])
    for anum,sattr in sel_attrs.items():
        attrsize = len(sattr.split(' '))
        anum,cnds,HH = get_attr_stats(anum,sattr,attrsize)
        hamming_steps2[(order,ss,rr,rep,anum)]=HH
        changing_nodes2[(order,ss,rr,rep,anum)]=cnds

for ii,nd_l in changing_nodes2.items():
    changing_node_sets2[ii]=' '.join(['%d' % nd for nd in nd_l])

#### Frequency of preservation under asynchronous dynamics of each attractor period

In [ ]:
### determination of attractors with db0
gn_tup_has_0dB0 = {}
preservation_freq_d = {}
for att_len,att_freq in att_sizes_ser.value_counts(normalize=True).items():
    if att_len==1:
        continue
    #print(att_len)
    periodx_d = {}
    for (srt, ss, rr, rep, attno)  in att_sizes_ser[att_sizes_ser==att_len].index:
        sattr = att_d[(srt, ss, rr, rep, attno)]
        attno,cnds,HH = get_attr_stats(attno,sattr,att_len)
        #sarr = np.asarray([[int(elt) for elt in bstr] for bstr in sattr.split(' ')])
        periodx_d[(srt, ss, rr, rep, attno)] = tuple(sorted([nodes[ii] for ii in cnds]))
    periodx_vcs = pd.Series(periodx_d).value_counts()
    #print(f'Period {att_len} value counts determined')
    tot_db0 = 0
    for gn_tup,numattr in periodx_vcs.items():
        dB0 = set([])
        for (gn1,gn2) in combs(gn_tup,2):
            dB0 |= set(list(G_rs2.successors(gn1)))&set(list(G_rs2.successors(gn2)))
        if len(set(dB0)-set(gn_tup))==0:
            tot_db0 +=numattr
            gn_tup_has_0dB0[tuple(sorted(list(gn_tup)))]=True
        else:
            gn_tup_has_0dB0[tuple(sorted(list(gn_tup)))]=False
    print(f'Period: {att_len}\t Frequency: {(tot_db0/periodx_vcs.sum()):.04f}')
    preservation_freq_d[att_len]=tot_db0/periodx_vcs.sum()
preservation_freq_ser = pd.Series(preservation_freq_d)    


#### Overall rate of attractor preservation under asynchronous dynamics

In [ ]:
print((attr_freq_ser*preservation_freq_ser).sum())

## Preservation of initial and final attractors for partial fixed points

In [ ]:
rr_ss_l = [(0.2,0.8),(0.8,0),(0.6,0),(0.4,0),(0.2,0),
           (1,0),(1,0.2),(1,0.4),(1,0.6),(1,0.8),
           (0.8,0.2),(0.6,0.4),(0.4,0.6),(0,1)]
stats_d = {}
pfp2fp_d = {}
pfp2fp_pres_d = {}
fp2pfp_d = {}
fp2pfp_pres_d = {}
pfp2pfp_d = {}
tot_pres = 0
tot_pfp_trans = 0
for srt in ['asc','desc']:
    print(srt)
    for (rr,ss) in rr_ss_l:
        print(rr,ss)
        ii_vals = range(20) if not (ss,rr) in [(0,1),(1,0),(1,1),(0,0)] else [0]
        for ii in ii_vals:
            print(ii)
            for pt in ['KO','OE']:
                fn = f'results/result-attr-trans-{pt}-twoparam_{ss:.02f}_{rr:.02f}_{srt}_{ii:02d}_{ii}.csv'
                kw_tup = (srt,ss,rr,ii,pt)
                row_d = {}
                try:
                    RES2 = pd.read_csv(fn)
                except FileNotFoundError:
                    print("missing file",fn)
                    continue
                cX = (RES2.init_attr_index==RES2.fin_attr_index)
                row_d['total_trans'] = cX.shape[0]
                row_d['not_act_irrev'] = cX.sum()
                RES = RES2[~cX]
                condA  = (RES.init_num_fixed==RES.fin_num_fixed) ## no. time-dependent nodes equal before/after
                condB = (RES.init_num_fixed>0) ## no. time-dependent nodes greater than 0 in initial attr
                condC = (RES.fin_num_fixed>0) ## no. time-dependent nodes greater than 0 in initial attr
                condD = (RES.num_fixed_changed>0) ## no. time-independent nodes that change between attractors
                pfp_trans = RES[(condA) & (condB)] ## partial fixed point attractors
                row_d['eq_attrsize']=pfp_trans.shape[0]
                pfp_trans_fixed = RES[(condA) & (condB)] ## transitions between PFPs w/ fixed part changed only
                row_d['eq_attrsize_fixonly']=pfp_trans_fixed.is_fixed_changed.sum()
                fp_trans = RES[(condA) & (~condB)] ## transitions between FPs
                row_d['fp_trans']=fp_trans.shape[0]
                fp2pfp_trans = RES[(~condA) & (condB&~condC)] ## transition FP -> PFP, look at the genes responsible
                fp2pfp_perts = fp2pfp_trans.pert_gene_name.value_counts()
                fp2pfp_d[kw_tup] = fp2pfp_perts
                fp2pfp_trans_pres = RES[(~condA) & (condB&~condC)&(condD)] ## transition FP -> PFP, look at the genes responsible
                fp2pfp_perts_pres = fp2pfp_trans_pres.pert_gene_name.value_counts()
                fp2pfp_pres_d[kw_tup] = fp2pfp_perts_pres
                pfp2fp_trans = RES[(~condA) & (~condB&condC)] ## transition PFP -> FP, look at the genes responsible
                pfp2fp_perts = pfp2fp_trans.pert_gene_name.value_counts()
                pfp2fp_d[kw_tup]=pfp2fp_perts
                pfp2fp_trans_pres = RES[(~condA) & (~condB&condC)&(condD)] ## transition PFP -> FP, look at the genes responsible
                pfp2fp_perts_pres = pfp2fp_trans_pres.pert_gene_name.value_counts()
                pfp2fp_pres_d[kw_tup]=pfp2fp_perts_pres
                pfp_trans_diff = RES[(~condA) & (condB & condC)] ## transitions between PFPs with different periods
                pfp2pfp_perts = pfp_trans_diff.pert_gene_name.value_counts()
                pfp2pfp_d[kw_tup]=pfp2pfp_perts
                row_d['pfp_neq_periods']=pfp_trans_diff.shape[0]
                for kk,vv in row_d.items():
                    stats_d[tuple(list(kw_tup)+[kk])]=vv
                
                
                pfp_vcs = pfp_trans.loc[:,['init_attr_index','fin_attr_index']].value_counts()
                for (iai,fai),tcnt in pfp_vcs.items():
                    tot_pfp_trans+=tcnt
                    iattr = att_ser.loc[(srt,ss,rr,ii,iai)]
                    iattrsize = len(iattr.split(' '))
                    iai,icnds,iHH = get_attr_stats(iai,iattr,iattrsize)
                    ign_tup = tuple(sorted([nodes[icnd] for icnd in icnds]))
                    if ign_tup in gn_tup_has_0dB0.keys():
                        iTF = gn_tup_has_0dB0[ign_tup]
                    else:
                        if iattrsize in [2,4]:
                            print(ign_tup)
                        iTF = False
                    fattr = att_ser.loc[(srt,ss,rr,ii,fai)]
                    fattrsize = len(fattr.split(' '))
                    fai,fcnds,fHH = get_attr_stats(fai,fattr,fattrsize)
                    fgn_tup = tuple(sorted([nodes[fcnd] for fcnd in fcnds]))
                    if fgn_tup in gn_tup_has_0dB0.keys():
                        fTF = gn_tup_has_0dB0[fgn_tup]
                    else:
                        if fattrsize in [2,4]:
                            print(fgn_tup)
                            break
                        fTF = False
                    if iTF and fTF: ## both initial and final pfps are preserved
                        tot_pres+=tcnt


#### Rate of preseravation of transitions between pfps


In [ ]:
print(tot_pres/tot_pfp_trans)

In [20]:
stats_df = pd.Series(stats_d).unstack()
stats_df2 = stats_df.groupby(level=[0,1,2]).sum()
tofromfp_d = {}
for kk,vv in pfp2fp_d.items():
    if kk in fp2pfp_d.keys():
        tofromfp_d[kk] = ((fp2pfp_d[kk].sum()+vv.sum())/stats_df.loc[kk,'total_trans'])
    else:
        tofromfp_d[kk] = vv.sum()/stats_df.loc[kk,'total_trans']
for kk in set(fp2pfp_d.keys())-set(pfp2fp_d.keys()):
    tofromfp_d[kk] = fp2pfp_d[kk].sum()/stats_df.loc[kk,'total_trans']
## Number of transitions in which only one is a fixed point    
pd.Series(tofromfp_d).mean()

In [ ]:
pfp2fp_d2 = dict(filter(lambda kk: kk[1].shape[0]>0,pfp2fp_d.items()))
fp2pfp_d2 = dict(filter(lambda kk: kk[1].shape[0]>0,fp2pfp_d.items()))
pfp2fp_pres_d2 = dict(filter(lambda kk: kk[1].shape[0]>0,pfp2fp_pres_d.items()))
fp2pfp_pres_d2 = dict(filter(lambda kk: kk[1].shape[0]>0,fp2pfp_pres_d.items()))

tmp_d ={}
for ind,row in stats_df2.iterrows():
    A = row.loc['fp_trans']/row.loc['total_trans']
    B = row.loc['eq_attrsize_fixonly']/row.loc['total_trans']
    pfp2fp_keys = [kk for kk in pfp2fp_d2.keys() if all([ind[ii]==kk[ii] for ii in range(3)])]
    fp2pfp_keys = [kk for kk in fp2pfp_d2.keys() if all([ind[ii]==kk[ii] for ii in range(3)])]
    stats_df_keys = [kk for kk in stats_df.index if all([ind[ii]==kk[ii] for ii in range(3)])]
    C_l = []
    for kk in list(set(stats_df_keys)):
        if kk in pfp2fp_keys and kk in fp2pfp_keys:
            CC = (pfp2fp_d2[kk].sum()+fp2pfp_d2[kk].sum())/stats_df.loc[kk,'total_trans']
        elif kk in pfp2fp_keys:
            CC = pfp2fp_d2[kk].sum()/stats_df.loc[kk,'total_trans']
        elif kk in fp2pfp_keys:
            CC = fp2pfp_d2[kk].sum()/stats_df.loc[kk,'total_trans']
        else:
            CC = 0
        C_l.append(CC)
    C = np.mean(C_l)
    pfp2fp_pres_keys = [kk for kk in pfp2fp_pres_d2.keys() if all([ind[ii]==kk[ii] for ii in range(3)])]
    fp2pfp_pres_keys = [kk for kk in fp2pfp_pres_d2.keys() if all([ind[ii]==kk[ii] for ii in range(3)])]
    stats_pres_df_keys = [kk for kk in stats_df.index if all([ind[ii]==kk[ii] for ii in range(3)])]
    D_l = []
    for kk in list(set(stats_pres_df_keys)):
        if kk in pfp2fp_pres_keys and kk in fp2pfp_pres_keys:
            DD = (pfp2fp_pres_d2[kk].sum()+fp2pfp_pres_d2[kk].sum())/stats_df.loc[kk,'total_trans']
        elif kk in pfp2fp_pres_keys:
            DD = pfp2fp_pres_d2[kk].sum()/stats_df.loc[kk,'total_trans']
        elif kk in fp2pfp_pres_keys:
            DD = fp2pfp_pres_d2[kk].sum()/stats_df.loc[kk,'total_trans']
        else:
            DD = 0
        D_l.append(DD)
    D = np.mean(D_l)
    tmp_d[tuple(list(ind)+['fp'])]=A
    tmp_d[tuple(list(ind)+['pfp'])]=B
    tmp_d[tuple(list(ind)+['tofromfp'])]=C
    tmp_d[tuple(list(ind)+['tofromfp_pres'])]=D
## Fraction of transitions of each type, including those that are preserved
transition_stats = pd.Series(tmp_d).unstack().mean()
transition_stats

In [ ]:
## rate of preserved attractors (transition types are taken from the table above)
transition_stats.loc['fp'] + transition_stats.loc['pfp'] * tot_pres/tot_pfp_trans + transition_stats.loc['tofromfp_pres']